In [ ]:
import numpy as np
import requests
r = requests.get("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a")
t = requests.get("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a.t")

def loss_function(x, y, w):
    x_shape = x.shape
    m = x_shape[0]
    xw = x@w
    return 1 / m * np.log(1 + np.exp(-1 * y.T.dot(x.dot(w)))) + 1/2 * (np.sum(w*w))

def grad(x, y, w):
    x_shape = x.shape
    m = x_shape[0]

    grad = np.random.rand(x_shape[1], 1)
    for i in range(0, x_shape[0]):
        yx = y[i] * (x[i:i + 1, :].T)
        grad = grad + yx / (1 + np.exp(y.T.dot(x.dot(w))))
    grad = w - 1 / m * grad
    return grad


def getY(x, w):
    return np.where(x@w>0,1,-1)

from io import BytesIO
from sklearn.datasets import load_svmlight_file

# get training set
X_train, Y_train = load_svmlight_file(BytesIO(r.content), n_features=123)
X_train = X_train.toarray()
Y_train = Y_train.reshape((-1, 1))
x_shape = X_train.shape
x0 = np.ones((x_shape[0], 1))
X_train = np.column_stack((X_train, x0))

W = np.zeros((x_shape[1] + 1, 1))

# get validation set
X_test, Y_test = load_svmlight_file(BytesIO(t.content), n_features=123)
X_test = X_test.toarray()
Y_test = Y_test.reshape((-1, 1))

xtest_shape = X_test.shape
x00 = np.ones((xtest_shape[0], 1))
X_test = np.column_stack((X_test, x00))

losses_val = []

import random

for count in range(0, 50):
    # each time choose 128 samples randomly
    i = random.randint(0, x_shape[0])
    x_sample = X_train[i:i + 128, :]
    y_sample = Y_train[i:i + 128, :]

    W = W - 0.00001 * grad(x_sample, y_sample, W)

    loss_val = loss_function(X_test, Y_test, W)
    losses_val.append(loss_val[0])

import matplotlib.pyplot as plt

plt.figure(figsize=(18, 6))
plt.plot(losses_val, "-", color="b", label="validation loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()
plt.title("The graph of absolute diff value varing with the number of iterations")